In [1]:
using Pkg
Pkg.build("PyCall")
Pkg.add("PyCall")
Pkg.add("ScikitLearn")
Pkg.add("CSV")
using PyCall, ScikitLearn, CSV, DataFrames
Pickle = pyimport("pickle")
@sk_import preprocessing: LabelEncoder;
@sk_import model_selection: train_test_split;
@sk_import naive_bayes: CategoricalNB;
@sk_import tree: DecisionTreeClassifier;
@sk_import metrics: accuracy_score;
@sk_import svm: SVC;
@sk_import neural_network: MLPClassifier;
@sk_import naive_bayes: GaussianNB;
@sk_import preprocessing: LabelBinarizer;
@sk_import svm: LinearSVC;

   Building Conda ─→ `~/.julia/packages/Conda/tJJuN/deps/build.log`
   Building PyCall → `~/.julia/packages/PyCall/tqyST/deps/build.log`
   Updating registry at `~/.julia/registries/General`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
┌ Warning: Module model_selection has been ported to Julia - try `import ScikitLearn: CrossValidation` instead
└ @ ScikitLearn.Skcore /Users/kevintram/.julia/packages/ScikitLearn/NJwUf/src/Skcore.jl:179


# Car Data Set

## Preprocessing

For all methods, the data was partitioned in a 70-30 split. All the features and labels were label encoded. 

In [2]:
car = CSV.File("car/car.data", header = ["buying", "maint", "doors", "persons", "lug_boot", "saftey", "acc"]) |> DataFrame

,buying,maint,doors,persons,lug_boot,saftey,acc
,String,String,String,String,String,String,String
1,vhigh,vhigh,2,2,small,low,unacc
2,vhigh,vhigh,2,2,small,med,unacc
3,vhigh,vhigh,2,2,small,high,unacc
4,vhigh,vhigh,2,2,med,low,unacc
5,vhigh,vhigh,2,2,med,med,unacc
6,vhigh,vhigh,2,2,med,high,unacc
7,vhigh,vhigh,2,2,big,low,unacc
8,vhigh,vhigh,2,2,big,med,unacc
9,vhigh,vhigh,2,2,big,high,unacc


Encode the data with a label encoder:

In [3]:
le = LabelEncoder();

In [4]:
for col = 1:7
    car[!, col] = fit_transform!(le, car[!, col]);
end
car

,buying,maint,doors,persons,lug_boot,saftey,acc
,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,3,3,0,0,2,1,2
2,3,3,0,0,2,2,2
3,3,3,0,0,2,0,2
4,3,3,0,0,1,1,2
5,3,3,0,0,1,2,2
6,3,3,0,0,1,0,2
7,3,3,0,0,0,1,2
8,3,3,0,0,0,2,2
9,3,3,0,0,0,0,2


Seperate data into independent and dependent variables:

In [5]:
car_x = convert(Array, car[:,collect(1:6)])

1728×6 Array{Int64,2}:
 3  3  0  0  2  1
 3  3  0  0  2  2
 3  3  0  0  2  0
 3  3  0  0  1  1
 3  3  0  0  1  2
 3  3  0  0  1  0
 3  3  0  0  0  1
 3  3  0  0  0  2
 3  3  0  0  0  0
 3  3  0  1  2  1
 3  3  0  1  2  2
 3  3  0  1  2  0
 3  3  0  1  1  1
 ⋮              ⋮
 1  1  3  1  0  1
 1  1  3  1  0  2
 1  1  3  1  0  0
 1  1  3  2  2  1
 1  1  3  2  2  2
 1  1  3  2  2  0
 1  1  3  2  1  1
 1  1  3  2  1  2
 1  1  3  2  1  0
 1  1  3  2  0  1
 1  1  3  2  0  2
 1  1  3  2  0  0

In [6]:
car_y = convert(Array, car[:,7])

1728-element Array{Int64,1}:
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 ⋮
 2
 1
 3
 2
 0
 1
 2
 1
 3
 2
 1
 3

Split into testing and training. 

In [7]:
X_train, X_test, y_train, y_test = train_test_split(car_x, car_y, test_size=0.30);

## Naive Bayes

No hyperparameter tuning was done. 

In [8]:
car_nb_model = CategoricalNB(); # because all data is categorical

In [9]:
@time car_nb_model.fit(X_train, y_train);

  0.013685 seconds (29.51 k allocations: 1.569 MiB)


In [10]:
@time predictions = predict(car_nb_model, X_test)

  0.022870 seconds (28.44 k allocations: 1.533 MiB)


519-element Array{Int64,1}:
 2
 3
 2
 0
 0
 2
 2
 2
 0
 2
 0
 2
 2
 ⋮
 2
 2
 2
 2
 2
 2
 2
 0
 2
 2
 2
 2

In [11]:
accuracy = accuracy_score(predictions, y_test)

0.8420038535645472

In [12]:
println("$(sizeof(Pickle.dumps(car_nb_model))) bytes")

2424 bytes


## Classification Tree

No hyperparameter tuning was done. 

In [13]:
car_tree_model = DecisionTreeClassifier();

In [14]:
@time car_tree_model.fit(X_train, y_train);

  0.004260 seconds (17 allocations: 976 bytes)


In [15]:
@time predictions = predict(car_tree_model, X_test)

  0.001047 seconds (36 allocations: 5.875 KiB)


519-element Array{Int64,1}:
 2
 3
 2
 1
 3
 2
 2
 2
 0
 2
 1
 2
 2
 ⋮
 2
 2
 2
 2
 2
 0
 2
 0
 2
 2
 2
 2

In [16]:
accuracy = accuracy_score(predictions, y_test)

0.9653179190751445

In [17]:
println("$(sizeof(Pickle.dumps(car_tree_model))) bytes")

14053 bytes


## Support Vector Machines

No hyperparameter tuning was done. 

In [18]:
car_svm_model = SVC();

In [19]:
@time car_svm_model.fit(X_train, y_train);

  0.072835 seconds (17 allocations: 976 bytes)


In [20]:
@time predictions = car_svm_model.predict(X_test)

  0.061201 seconds (36 allocations: 5.875 KiB)


519-element Array{Int64,1}:
 2
 3
 2
 0
 3
 2
 2
 2
 0
 2
 0
 2
 2
 ⋮
 2
 2
 2
 2
 2
 2
 2
 0
 2
 2
 2
 2

In [21]:
accuracy = accuracy_score(predictions, y_test)

0.8901734104046243

In [22]:
println("$(sizeof(Pickle.dumps(car_svm_model))) bytes")

49596 bytes


## Neural Networks

No hyperparameter tuning was done. 

In [23]:
car_nn_model = MLPClassifier();

In [24]:
@time car_nn_model.fit(X_train, y_train);

  1.114169 seconds (20 allocations: 2.328 KiB)


In [25]:
@time predictions = car_nn_model.predict(X_test)

  0.016726 seconds (36 allocations: 5.875 KiB)


519-element Array{Int64,1}:
 2
 3
 2
 0
 3
 2
 2
 2
 0
 2
 0
 2
 2
 ⋮
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2

In [26]:
accuracy = accuracy_score(predictions, y_test)

0.882466281310212

In [27]:
println("$(sizeof(Pickle.dumps(car_nn_model))) bytes")

34776 bytes


# Abalone Data Set

## Preprocessing:

For all methods, the data was partitioned into a 70-30 training-testing split. The sex column was one hot encoded. The labels were grouped into three categories: 1-8, 9-10, 11+, for better classification results (this was used in paper, Clark, Schreter, and Adams 1996) 

In [28]:
abalone = CSV.File(
    "abalone/abalone.data", 
    header=["Sex", "Length", "Diameter", "Height", "Whole wt", "Shucked wt", "Viscera wt", "Shucked wt", "Rings"]
    ) |> DataFrame

,Sex,Length,Diameter,Height,Whole wt,Shucked wt,Viscera wt,Shucked wt_1
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,M,0.455,0.365,0.095,0.514,0.2245,0.101,0.15
2,M,0.35,0.265,0.09,0.2255,0.0995,0.0485,0.07
3,F,0.53,0.42,0.135,0.677,0.2565,0.1415,0.21
4,M,0.44,0.365,0.125,0.516,0.2155,0.114,0.155
5,I,0.33,0.255,0.08,0.205,0.0895,0.0395,0.055
6,I,0.425,0.3,0.095,0.3515,0.141,0.0775,0.12
7,F,0.53,0.415,0.15,0.7775,0.237,0.1415,0.33
8,F,0.545,0.425,0.125,0.768,0.294,0.1495,0.26
9,M,0.475,0.37,0.125,0.5095,0.2165,0.1125,0.165


Perform one hot encoding on sex:

In [29]:
lb = LabelBinarizer();

In [30]:
sex_encoded = fit_transform!(lb, abalone[!,1]);

In [31]:
select!(abalone, Not(:Sex)); # remove sex column

In [32]:
insertcols!(abalone, 1, :M => sex_encoded[:,1], :F => sex_encoded[:,2], :I => sex_encoded[:,3]); # append encoded columns

Split data set into X and y

In [33]:
abalone_x = convert(Array, abalone[:,collect(1:10)])

4177×10 Array{Float64,2}:
 0.0  0.0  1.0  0.455  0.365  0.095  0.514   0.2245  0.101   0.15
 0.0  0.0  1.0  0.35   0.265  0.09   0.2255  0.0995  0.0485  0.07
 1.0  0.0  0.0  0.53   0.42   0.135  0.677   0.2565  0.1415  0.21
 0.0  0.0  1.0  0.44   0.365  0.125  0.516   0.2155  0.114   0.155
 0.0  1.0  0.0  0.33   0.255  0.08   0.205   0.0895  0.0395  0.055
 0.0  1.0  0.0  0.425  0.3    0.095  0.3515  0.141   0.0775  0.12
 1.0  0.0  0.0  0.53   0.415  0.15   0.7775  0.237   0.1415  0.33
 1.0  0.0  0.0  0.545  0.425  0.125  0.768   0.294   0.1495  0.26
 0.0  0.0  1.0  0.475  0.37   0.125  0.5095  0.2165  0.1125  0.165
 1.0  0.0  0.0  0.55   0.44   0.15   0.8945  0.3145  0.151   0.32
 1.0  0.0  0.0  0.525  0.38   0.14   0.6065  0.194   0.1475  0.21
 0.0  0.0  1.0  0.43   0.35   0.11   0.406   0.1675  0.081   0.135
 0.0  0.0  1.0  0.49   0.38   0.135  0.5415  0.2175  0.095   0.19
 ⋮                            ⋮                              
 0.0  1.0  0.0  0.405  0.3    0.085  0.3035  0.15 

In [34]:
abalone_y = convert(Array, abalone[:,11])

4177-element Array{Int64,1}:
 15
  7
  9
 10
  7
  8
 20
 16
  9
 19
 14
 10
 11
  ⋮
  7
 10
  9
  8
 10
 10
  8
 11
 10
  9
 10
 12

Transform y into three categories, 1-8, 9-10, 11+, for better classification results (this was also used in Clark, Schreter, and Adams 1996)

In [35]:
for i = 1:length(abalone_y)
    val = abalone_y[i]
    if (val >= 1 && val <= 8)
        abalone_y[i] = 1
    elseif (val >= 9 && val <= 10)
        abalone_y[i] = 2
    else
        abalone_y[i] = 3
    end
end
abalone_y

4177-element Array{Int64,1}:
 3
 1
 2
 2
 1
 1
 3
 3
 2
 3
 3
 2
 3
 ⋮
 1
 2
 2
 1
 2
 2
 1
 3
 2
 2
 2
 3

Split into training and testing:

In [36]:
X_train, X_test, y_train, y_test = train_test_split(abalone_x, abalone_y, test_size=0.30);

## Naive Bayes

I used a Gaussian Naive Bayes classifier since most of the data were continous values. 

In [37]:
abalone_nb_model = GaussianNB();

In [38]:
@time abalone_nb_model.fit(X_train, y_train);

  0.027623 seconds (29.50 k allocations: 1.567 MiB)


In [39]:
@time predictions = abalone_nb_model.predict(X_test)

  0.015612 seconds (25.01 k allocations: 1.351 MiB)


1254-element Array{Int64,1}:
 3
 3
 2
 2
 1
 1
 3
 2
 2
 2
 2
 3
 1
 ⋮
 1
 3
 1
 3
 3
 2
 2
 2
 2
 2
 2
 2

In [40]:
accuracy = accuracy_score(predictions, y_test)

0.5701754385964912

In [41]:
println("$(sizeof(Pickle.dumps(abalone_nb_model))) bytes")

1084 bytes


## Classification Tree

No hyperparameter tuning was done here. 

In [42]:
abalone_tree_model = DecisionTreeClassifier();

In [43]:
@time abalone_tree_model.fit(X_train, y_train);

  0.041816 seconds (17 allocations: 976 bytes)


In [44]:
@time predictions = abalone_tree_model.predict(X_test)

  0.000516 seconds (36 allocations: 11.625 KiB)


1254-element Array{Int64,1}:
 3
 3
 2
 2
 1
 1
 2
 2
 1
 3
 1
 2
 2
 ⋮
 1
 2
 2
 2
 2
 2
 2
 2
 3
 2
 1
 2

In [45]:
accuracy = accuracy_score(predictions, y_test)

0.5606060606060606

In [46]:
println("$(sizeof(Pickle.dumps(abalone_tree_model))) bytes")

121361 bytes


## Support Vector Machine

No hyper parameter tuning was done here. 

In [47]:
abalone_svm_model = SVC();

In [48]:
@time abalone_svm_model.fit(X_train, y_train);

  0.364268 seconds (17 allocations: 976 bytes)


In [49]:
@time predictions = abalone_svm_model.predict(X_test)

  0.480712 seconds (36 allocations: 11.625 KiB)


1254-element Array{Int64,1}:
 2
 3
 2
 2
 1
 2
 3
 2
 2
 2
 2
 3
 1
 ⋮
 1
 3
 1
 2
 2
 3
 2
 2
 3
 3
 2
 3

In [50]:
accuracy = accuracy_score(predictions, y_test)

0.6427432216905901

In [51]:
println("$(sizeof(Pickle.dumps(abalone_svm_model))) bytes")

218201 bytes


## Neural Network

No hyper parameter tuning was done here. 

In [52]:
abalone_nn_model = MLPClassifier();

In [53]:
@time abalone_nn_model.fit(X_train, y_train);

  2.312146 seconds (17 allocations: 976 bytes)


In [54]:
@time predictions = abalone_nn_model.predict(X_test)

  0.004024 seconds (36 allocations: 11.625 KiB)


1254-element Array{Int64,1}:
 2
 3
 2
 2
 1
 1
 3
 2
 2
 2
 1
 3
 1
 ⋮
 1
 3
 1
 2
 2
 3
 2
 2
 3
 3
 2
 3

In [55]:
accuracy = accuracy_score(predictions, y_test)

0.6499202551834131

In [56]:
println("$(sizeof(Pickle.dumps(abalone_nn_model))) bytes")

41944 bytes


# Madelon Data Set

## Preprocessing

For all methods, the data was partitioned into a 70-30 training-testing split. The validation data set also used for testing since the training data did not have labels.

In [57]:
X_train = CSV.File("madelon/madelon_train.data", delim=" ", header=false, type=Int16) |> DataFrame
select!(X_train, Not(:Column501)) # remove 501st column because there's an extra space at the end of each line
X_train = convert(Array, X_train)

2000×500 Array{Int16,2}:
 485  477  537  479  452  471  491  476  …  485  511  485  481  479  475  496
 483  458  460  487  587  475  526  479     338  513  486  483  492  510  517
 487  542  499  468  448  471  442  478     650  506  501  480  489  499  498
 480  491  510  485  495  472  417  474     572  454  469  475  482  494  461
 484  502  528  489  466  481  402  478     435  486  508  481  504  495  511
 481  496  451  480  516  484  484  476  …  435  491  564  482  463  457  500
 484  533  498  466  577  482  471  477     648  484  475  479  475  462  484
 474  468  598  490  530  482  448  476     351  462  533  480  501  529  489
 484  498  558  489  508  478  471  478     633  453  556  458  473  531  440
 496  448  570  476  477  481  595  477     576  511  508  478  486  582  496
 478  446  457  475  468  475  482  479  …  491  512  476  471  501  550  517
 486  479  528  483  530  484  479  477     599  459  452  475  496  505  485
 466  486  540  492  500  475  491  474

In [58]:
X_test = CSV.File("madelon/madelon_valid.data", delim=" ", header=false, type=Int16) |> DataFrame
select!(X_test, Not(:Column501)) # remove 501st column because there's an extra space at the end of each line
X_test = convert(Array, X_test)

600×500 Array{Int16,2}:
 483  454  513  495  523  469  453  477  …  259  413  520  485  498  523  510
 485  508  493  487  478  472  504  476     534  514  452  484  495  548  477
 483  521  507  475  493  486  421  475     495  508  528  486  465  508  503
 474  504  576  480  553  483  524  478     463  509  525  479  467  552  517
 495  474  523  479  495  488  485  476     343  509  520  475  493  506  491
 474  507  504  495  428  490  498  478  …  503  410  573  477  461  532  520
 481  475  509  478  505  489  499  478     351  489  488  487  465  494  519
 486  445  444  468  422  484  464  477     715  492  394  475  474  446  481
 482  483  525  477  452  480  488  477     557  449  490  484  490  476  466
 487  494  498  480  443  486  418  477     417  532  469  470  480  505  529
 481  513  513  464  527  462  510  476  …  409  478  447  478  488  508  467
 480  487  513  500  538  476  455  474     484  520  505  482  493  519  522
 470  436  452  490  549  476  441  477 

In [59]:
y_train = CSV.File("madelon/madelon_train.labels", header=false, type=Int8) |> DataFrame
y_train = convert(Array, y_train)

2000×1 Array{Int8,2}:
 -1
 -1
 -1
  1
  1
  1
  1
 -1
  1
 -1
 -1
 -1
  1
  ⋮
 -1
  1
 -1
 -1
 -1
 -1
  1
  1
 -1
 -1
  1
  1

In [60]:
y_test = CSV.File("madelon/madelon_valid.labels", header=false, type=Int8) |> DataFrame
y_test = convert(Array, y_test)

600×1 Array{Int8,2}:
 -1
 -1
 -1
  1
 -1
  1
 -1
 -1
 -1
  1
  1
  1
 -1
  ⋮
  1
 -1
 -1
  1
 -1
 -1
  1
  1
  1
  1
  1
 -1

## Naive Bayes

I used a gaussian naive bayes classifier since all the features were continuous values. 

In [61]:
madelone_nb_model = GaussianNB();

In [62]:
@time madelone_nb_model.fit(X_train, y_train);

  0.243823 seconds (337.92 k allocations: 16.435 MiB)


In [63]:
@time predictions = madelone_nb_model.predict(X_test)

  0.202003 seconds (452.75 k allocations: 24.063 MiB)


600-element Array{Int8,1}:
  1
 -1
 -1
  1
  1
  1
 -1
 -1
 -1
  1
  1
  1
  1
  ⋮
 -1
 -1
 -1
  1
  1
 -1
 -1
 -1
 -1
  1
  1
 -1

In [64]:
accuracy = accuracy_score(predictions, y_test)

0.5916666666666667

In [65]:
println("$(sizeof(Pickle.dumps(madelone_nb_model))) bytes")

16577 bytes


## Classification Tree

No hyperparameters were tuned here. 

In [66]:
madelone_tree_model = DecisionTreeClassifier();

In [67]:
@time madelone_tree_model.fit(X_train, y_train);

  0.545177 seconds (17 allocations: 976 bytes)


In [68]:
@time predictions = madelone_tree_model.predict(X_test)

  0.001536 seconds (55 allocations: 3.562 KiB)


600-element Array{Int8,1}:
  1
 -1
 -1
  1
 -1
  1
  1
 -1
  1
  1
  1
  1
  1
  ⋮
  1
 -1
 -1
  1
 -1
 -1
 -1
  1
 -1
  1
  1
 -1

In [69]:
accuracy = accuracy_score(predictions, y_test)

0.77

In [70]:
println("$(sizeof(Pickle.dumps(madelone_tree_model))) bytes")

22800 bytes


## Support Vector Machine

No hyperparameters were tuned here. 

In [71]:
madelone_svm_model = SVC();

In [72]:
@time madelone_svm_model.fit(X_train, y_train);

  1.463283 seconds (17 allocations: 976 bytes)


In [73]:
@time predictions = madelone_svm_model.predict(X_test)

  0.558318 seconds (55 allocations: 3.562 KiB)


600-element Array{Int8,1}:
 -1
 -1
 -1
  1
 -1
 -1
 -1
 -1
  1
  1
  1
  1
  1
  ⋮
 -1
 -1
 -1
  1
 -1
 -1
  1
 -1
 -1
  1
  1
 -1

In [74]:
accuracy = accuracy_score(predictions, y_test)

0.6866666666666666

In [75]:
println("$(sizeof(Pickle.dumps(madelone_svm_model))) bytes")

7301494 bytes


## Neural Network

No hyperparameters were turned here. 

In [76]:
madelone_nn_model = MLPClassifier();

In [77]:
@time madelone_nn_model.fit(X_train, y_train);

  1.219762 seconds (17 allocations: 976 bytes)


In [78]:
@time predictions = madelone_nn_model.predict(X_test)

  0.004404 seconds (55 allocations: 3.562 KiB)


600-element Array{Int8,1}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

In [79]:
accuracy = accuracy_score(predictions, y_test)

0.5116666666666667

In [80]:
println("$(sizeof(Pickle.dumps(madelone_nn_model))) bytes")

1209848 bytes


# KDD Data Set

## Preprocessing

For the Naive Bayes and Classification Tree methods, the data was partioned into a 70-30 split using only the 10% data set (since the full dataset would cause my laptop to run out of memory as well as cause the methods to take much longer). For the Support Vector Machine and Neural Network methods, the data was also partioned into a 70-30 split using only the 10% data set but it was further shrunken by only using 30% of the partitioned data, since any larger would cause out of memory errors and for methods to take much longer to predict and train. 
For all 4 methods, the protocol_type, service, and flag columns were label encoded. 

In [81]:
kdd = CSV.File("kdd/kddcup.data_10_percent",
    header = [
        "duration", "protocol_type", "service", "flag", "src_bytes", "dst_bytes", "land", 
        "wrong_fragment", "urgent", "hot", "num_failed_logins", "logged_in", "num_compromised", 
        "root_shell", "su_attempted", "num_root", "num_file_creations", "num_shells", 
        "num_access_files", "num_outbound_cmds", "is_hot_login","is_guest_login", "count",
        "srv_count", "serror_rate", "srv_serror_rate", "rerror_rate","srv_rerror_rate", 
        "same_srv_rate", "diff_srv_rate","srv_diff_host_rate", "dst_host_count",
        "dst_host_srv_count","dst_host_same_srv_rate", "dst_host_diff_srv_rate","
        dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate", "dst_host_serror_rate",
        "dst_host_srv_serror_rate", "dst_host_rerror_rate", "dst_host_srv_rerror_rate", "result"
    ]) |> DataFrame

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment
,Int64,String,String,String,Int64,Int64,Int64,Int64
1,0,tcp,http,SF,181,5450,0,0
2,0,tcp,http,SF,239,486,0,0
3,0,tcp,http,SF,235,1337,0,0
4,0,tcp,http,SF,219,1337,0,0
5,0,tcp,http,SF,217,2032,0,0
6,0,tcp,http,SF,217,2032,0,0
7,0,tcp,http,SF,212,1940,0,0
8,0,tcp,http,SF,159,4087,0,0
9,0,tcp,http,SF,210,151,0,0


Encode Categoritcal Data:

In [82]:
le = LabelEncoder();

In [83]:
for col = [2, 3, 4, 42]
    kdd[!, col] = fit_transform!(le, kdd[!, col]);
end
kdd

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,0,1,22,9,181,5450,0,0
2,0,1,22,9,239,486,0,0
3,0,1,22,9,235,1337,0,0
4,0,1,22,9,219,1337,0,0
5,0,1,22,9,217,2032,0,0
6,0,1,22,9,217,2032,0,0
7,0,1,22,9,212,1940,0,0
8,0,1,22,9,159,4087,0,0
9,0,1,22,9,210,151,0,0


Split independent and dependent variables:

In [84]:
kdd_x = convert(Array{Float32,2}, kdd[:, 1:41])

494021×41 Array{Float32,2}:
 0.0  1.0  22.0  9.0  181.0  5450.0  0.0  …  0.11  0.0   0.0   0.0   0.0  0.0
 0.0  1.0  22.0  9.0  239.0   486.0  0.0     0.05  0.0   0.0   0.0   0.0  0.0
 0.0  1.0  22.0  9.0  235.0  1337.0  0.0     0.03  0.0   0.0   0.0   0.0  0.0
 0.0  1.0  22.0  9.0  219.0  1337.0  0.0     0.03  0.0   0.0   0.0   0.0  0.0
 0.0  1.0  22.0  9.0  217.0  2032.0  0.0     0.02  0.0   0.0   0.0   0.0  0.0
 0.0  1.0  22.0  9.0  217.0  2032.0  0.0  …  0.02  0.0   0.0   0.0   0.0  0.0
 0.0  1.0  22.0  9.0  212.0  1940.0  0.0     1.0   0.04  0.0   0.0   0.0  0.0
 0.0  1.0  22.0  9.0  159.0  4087.0  0.0     0.09  0.04  0.0   0.0   0.0  0.0
 0.0  1.0  22.0  9.0  210.0   151.0  0.0     0.12  0.04  0.0   0.0   0.0  0.0
 0.0  1.0  22.0  9.0  212.0   786.0  0.0     0.12  0.05  0.0   0.0   0.0  0.0
 0.0  1.0  22.0  9.0  210.0   624.0  0.0  …  0.06  0.05  0.0   0.0   0.0  0.0
 0.0  1.0  22.0  9.0  177.0  1985.0  0.0     0.04  0.04  0.0   0.0   0.0  0.0
 0.0  1.0  22.0  9.0  222.0   773.0 

In [85]:
kdd_y = convert(Array{Int8,1}, kdd[:, 42])

494021-element Array{Int8,1}:
 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
  ⋮
 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
 11
 11

Split into test and train:

In [86]:
X_train, X_test, y_train, y_test = train_test_split(kdd_x, kdd_y, test_size=0.30);

## Naive Bayes

No hyperparamters were tuned here.

In [87]:
kdd_nb_model = GaussianNB();

In [88]:
@time kdd_nb_model.fit(X_train, y_train);

  0.286442 seconds (29.50 k allocations: 1.567 MiB)


In [89]:
@time predictions = kdd_nb_model.predict(X_test)

  1.293266 seconds (25.03 k allocations: 1.483 MiB)


148207-element Array{Int8,1}:
 11
 18
 18
 18
 11
 11
 18
 18
 18
 18
 18
 18
  9
  ⋮
 10
  9
  9
 18
 18
  9
  9
 11
 11
 18
 11
 18

In [90]:
accuracy = accuracy_score(predictions, y_test)

0.9380393638627056

In [91]:
println("$(sizeof(Pickle.dumps(kdd_nb_model))) bytes")

16019 bytes


## Classification Tree

No hyperparameters were tuned here.

In [92]:
kdd_tree_model = DecisionTreeClassifier();

In [93]:
@time kdd_tree_model.fit(X_train, y_train);

  2.044986 seconds (17 allocations: 976 bytes)


In [94]:
@time predictions = kdd_tree_model.predict(X_test)

  0.025362 seconds (56 allocations: 147.672 KiB)


148207-element Array{Int8,1}:
 11
 18
 18
 18
 11
 11
 18
 18
 18
 18
 18
 18
  9
  ⋮
 11
  9
  9
 18
 18
  9
  9
 11
 11
 18
 11
 18

In [95]:
accuracy = accuracy_score(predictions, y_test)

0.9995749188634815

In [96]:
println("$(sizeof(Pickle.dumps(kdd_tree_model))) bytes")

121394 bytes


## Preprocessing for SVM and NN methods

SVM and NN used 30% of already partitioned and split testing and training data.

In [97]:
test_rows_cut = convert(Int, floor(length(y_test) * .30)) # use 30% of data

44462

In [98]:
X_test = X_test[1:test_rows_cut, :]

44462×41 Array{Float32,2}:
    0.0  1.0  22.0  9.0   312.0   1176.0  …  0.04  0.04  0.0  0.0  0.0   0.0
    0.0  0.0  14.0  9.0  1032.0      0.0     1.0   0.0   0.0  0.0  0.0   0.0
    0.0  0.0  14.0  9.0  1032.0      0.0     1.0   0.0   0.0  0.0  0.0   0.0
    0.0  0.0  14.0  9.0  1032.0      0.0     1.0   0.0   0.0  0.0  0.0   0.0
    0.0  1.0  22.0  9.0   159.0   3752.0     0.0   0.01  0.0  0.0  0.04  0.04
   64.0  1.0  56.0  9.0   166.0  15661.0  …  0.01  0.0   0.0  0.0  0.0   0.0
    0.0  0.0  14.0  9.0  1032.0      0.0     1.0   0.0   0.0  0.0  0.0   0.0
    0.0  0.0  14.0  9.0  1032.0      0.0     1.0   0.0   0.0  0.0  0.0   0.0
    0.0  0.0  14.0  9.0  1032.0      0.0     1.0   0.0   0.0  0.0  0.0   0.0
    0.0  0.0  14.0  9.0  1032.0      0.0     1.0   0.0   0.0  0.0  0.0   0.0
    0.0  0.0  14.0  9.0  1032.0      0.0  …  1.0   0.0   0.0  0.0  0.0   0.0
    0.0  0.0  14.0  9.0  1032.0      0.0     1.0   0.0   0.0  0.0  0.0   0.0
    0.0  1.0  45.0  1.0     0.0      0.0     0.0

In [99]:
y_test = y_test[1:test_rows_cut]

44462-element Array{Int8,1}:
 11
 18
 18
 18
 11
 11
 18
 18
 18
 18
 18
 18
  9
  ⋮
 18
 11
 18
  9
 18
  9
 11
  9
 18
 18
 11
 11

In [100]:
train_rows_cut = convert(Int, floor(length(y_train) * .30)) # use 30% of data

103744

In [101]:
X_train = X_train[1:train_rows_cut, :]

103744×41 Array{Float32,2}:
    0.0  0.0  14.0  9.0  1032.0     0.0  …  1.0   0.0   0.0  0.0  0.0  0.0
    0.0  1.0  45.0  5.0     0.0     0.0     0.0   0.0   1.0  1.0  0.0  0.0
    0.0  0.0  14.0  9.0  1032.0     0.0     1.0   0.0   0.0  0.0  0.0  0.0
    0.0  0.0  14.0  9.0  1032.0     0.0     1.0   0.0   0.0  0.0  0.0  0.0
    0.0  1.0  22.0  9.0   210.0  1856.0     0.0   0.0   0.0  0.0  0.0  0.0
    0.0  1.0  45.0  5.0     0.0     0.0  …  0.0   0.0   1.0  1.0  0.0  0.0
    0.0  1.0  45.0  5.0     0.0     0.0     0.0   0.0   1.0  1.0  0.0  0.0
    0.0  0.0  14.0  9.0  1032.0     0.0     1.0   0.0   0.0  0.0  0.0  0.0
    0.0  0.0  14.0  9.0  1032.0     0.0     1.0   0.0   0.0  0.0  0.0  0.0
    0.0  1.0  19.0  9.0  3676.0     0.0     0.5   0.0   0.0  0.0  0.0  0.0
    0.0  1.0  22.0  9.0   303.0  1263.0  …  0.0   0.0   0.0  0.0  0.0  0.0
    0.0  0.0  14.0  9.0  1032.0     0.0     1.0   0.0   0.0  0.0  0.0  0.0
    0.0  0.0  14.0  9.0  1032.0     0.0     1.0   0.0   0.0  0.0  0.0  0

In [102]:
y_train = y_train[1:train_rows_cut, :]

103744×1 Array{Int8,2}:
 18
  9
 18
 18
 11
  9
  9
 18
 18
 11
 11
 18
 18
  ⋮
 18
 18
 18
 18
 18
  9
 11
  9
 11
 18
 18
 11

## Support Vector Machine

A linear support vector machine was used for better training times. 

In [103]:
kdd_svm_model = LinearSVC();

In [104]:
@time kdd_svm_model.fit(X_train, y_train);

 94.767732 seconds (25.12 k allocations: 1.347 MiB)


In [105]:
@time predictions = kdd_svm_model.predict(X_test)

  0.035781 seconds (56 allocations: 46.359 KiB)


44462-element Array{Int8,1}:
 11
 18
 18
 18
 11
 11
 18
 18
 18
 18
 18
 18
  9
  ⋮
 18
 11
 18
  9
 18
  9
 11
  9
 18
 18
 11
 11

In [106]:
accuracy = accuracy_score(predictions, y_test)

0.9852458278979803

In [107]:
println("$(sizeof(Pickle.dumps(kdd_svm_model))) bytes")

8047 bytes


## Neural Network

No hyperparamters were tuned here.

In [108]:
kdd_nn_model = MLPClassifier();

In [109]:
@time kdd_nn_model.fit(X_train, y_train);

 27.826168 seconds (17 allocations: 976 bytes)


In [110]:
@time predictions = kdd_nn_model.predict(X_test)

  0.068699 seconds (56 allocations: 46.359 KiB)


44462-element Array{Int8,1}:
 11
 18
 18
 18
 11
 11
 18
 18
 18
 18
 18
 18
  9
  ⋮
 18
 11
 18
  9
 18
  9
 11
  9
 18
 18
 11
 11

In [111]:
accuracy = accuracy_score(predictions, y_test)

0.9972111016148621

In [112]:
println("$(sizeof(Pickle.dumps(kdd_nn_model))) bytes")

82337 bytes
